In [ ]:
#this is for the response obtained by manually tuning the model in matlab, not the manually iterated physical system

In [1]:
# import pandas as pd
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import control as ct

In [3]:
%matplotlib

Using matplotlib backend: <object object at 0x000002712AD32A50>


In [25]:
#model response
m = 0.023
R = 0.02
g = 9.8
L = 0.20
d = 0.12
J = (2/3)*m*R**2
s = ct.tf('s')
G = ((m*g*d)/(L*((J/R**2)+m)))*(1/s**2)
print(G)

Kp = 6
Ki = 0.0005
Kd = 7

C = Kp+Ki/s+Kd*s
plant = ct.feedback(G*C)
print(plant)
T,U = ct.step_response(15*plant,T = 2)
fig8,ax8 = plt.subplots()

L = len(U)
zeros = []
for i in range(0,L-1):
    if abs((U[-1]-U[i-1])/U[-1])>0.05 and abs((U[-1]-U[i+1])/U[-1])<0.05:
        zeros.append(T[i])
for i in range(0,L):
    if (U[-1]-U[i])/U[-1]<0.05 and T[i]>=zeros[-1]:
        I = i
        sett=T[i]
        Usett = U[i]
        break

dict = ct.step_info(15*plant)
print(dict)
over = dict['Overshoot']
print('Overshoot = ',over,'%')
rise = dict['RiseTime']
print('Rise time =',rise,'s')
print("Settling time =",sett,'s')

ax8.plot(T,U,label = 'response')
ax8.axhline(15,color = 'red',linestyle ='dotted',label = 'setpoint')
ax8.axhline(U[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax8.axhline(U[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax8.grid()
ax8.plot(sett,Usett,'*',label = "settling point",color = 'r')
ax8.vlines(T[np.argmax(U)],ymin=15,ymax = np.max(U),color='r',label = "overshoot")
ax8.legend()
ax8.set_ylabel('Displacement [cm]',fontsize=15)
ax8.set_xlabel('Time [s]',fontsize=15)
fig8.suptitle('Manually Tuned Model Step Response for Squash Ball',fontsize=15)




3.528
-----
 s^2


   24.7 s^2 + 21.17 s + 0.001764
-----------------------------------
s^3 + 24.7 s^2 + 21.17 s + 0.001764

{'RiseTime': 0.08206498538521813, 'SettlingTime': nan, 'SettlingMin': 13.617184514329228, 'SettlingMax': 15.434099850482726, 'Overshoot': 2.893999003218172, 'Undershoot': 0, 'Peak': 15.434099850482726, 'PeakTime': 0.28722744884826346, 'SteadyStateValue': 15.0}
Overshoot =  2.893999003218172 %
Rise time = 0.08206498538521813 s
Settling time = 0.11014492753623188 s


Text(0.5, 0.98, 'Manually Tuned Model Step Response for Squash Ball')

In [23]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(5,2,figsize=(9,13))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
plt.subplot_tool()
figraw,axsraw = plt.subplots(5,2,figsize=(9,13))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figz,axz = plt.subplots(5,2,figsize = (9,13))
plt.subplots_adjust(top = 0.95,bottom = 0.05)

fig.suptitle('Individual Manually Tuned Responses for Squash Ball',fontsize=15)
figraw.suptitle('Unfiltered Manually Tuned Responses for Squash Ball',fontsize=15)
figz.suptitle('Motor Angle for Manually Tuned Squash Ball Response',fontsize=15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Manually Tuned Squash Ball Responses',fontsize=15)
#fig3.suptitle('Unfiltered Average of Manually Tuned Squash Ball Responses')
ax2.set_xlabel('Time [s]',fontsize=15)
ax2.set_ylabel('Distance [cm]',fontsize=15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(4, 800, 'lp', fs=50000, output='sos')
sosang = signal.butter(4, 10000, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
ticksang = np.linspace(-40,40,17)
END = 1200
for i in range(1,6):
    df = pd.read_csv('SQUASH_MAT_MAN_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sosang,angle)
    
    axs[i-1,0].plot(timenew,filtered,label = 'filtered',color = 'b')
    axs[i-1,0].plot(timenew,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs[i-1,0].set_xlabel('Time [s]')
    axs[i-1,0].set_ylabel('Distance [cm]')
    axs[i-1,0].axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
    axs[i-1,0].grid()
    
    axsraw[i-1,0].plot(timenew,distance)
    axsraw[i-1,0].set_xlabel('Time [s]')
    axsraw[i-1,0].set_ylabel('Distance [cm]')
    axsraw[i-1,0].grid()
    
    axz[i-1,0].plot(timenew,angle_filt,label = 'filtered',color = 'b')
    axz[i-1,0].plot(timenew,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz[i-1,0].set_xlabel('Time [s]')
    axz[i-1,0].set_ylabel('Servo Angle [degrees]')
    axz[i-1,0].grid()
    axz[i-1,0].axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    axz[i-1,0].set_yticks(ticksang)

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
for i in range(1,6):
    df = pd.read_csv('SQUASH_MAT_MAN_{}.csv'.format(i+5))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sosang,angle)
    
    axs[i-1,1].plot(timenew,filtered,label = 'filtered',color = 'b')
    axs[i-1,1].plot(timenew,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs[i-1,1].set_xlabel('Time [s]')
    axs[i-1,1].set_ylabel('Distance [cm]')
    axs[i-1,1].axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
    axs[i-1,1].grid()
    
    axsraw[i-1,1].plot(timenew,distance)
    axsraw[i-1,1].set_xlabel('Time [s]')
    axsraw[i-1,1].set_ylabel('Distance [cm]')
    axsraw[i-1,1].grid()
    
    axz[i-1,1].plot(timenew,angle_filt,label = 'filtered',color ='b')
    axz[i-1,1].plot(timenew,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz[i-1,1].set_xlabel('Time [s]')
    axz[i-1,1].set_ylabel('Servo Angle [degrees]')
    axz[i-1,1].grid()
    axz[i-1,1].axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    axz[i-1,1].set_yticks(ticksang)
    time_name =f"time_{i+5}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i+5}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
axs[0,0].legend()
axz[0,0].legend()
# for i in range(0,END):
#     time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
#     dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
# print(time_ave)
# filtered = signal.sosfiltfilt(sos,dist_ave)
# L = len(filtered)
# zeros = []
# for i in range(0,L-1):
#     if time_ave[i]<2.5 and filtered[i]>filtered[i-1]:
#         OS = filtered[i]
#         OST = time_ave[i]
# # OS = np.max(filtered)
# # OSind = np.argmax(filtered)
# # OST = time_ave[OSind]

# for i in range(0,L-1):
#     if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
#         zeros.append(time_ave[i])
# for i in range(0,L):
#     if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
#         I = i
#         sett=time_ave[i]
#         Usett = filtered[i]
#         break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = "darkslategrey")
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right',fontsize=15)
ticks = np.linspace(0,23,24)
ax2.set_yticks(ticks)
#ax3.plot(time_ave,dist_ave,label = 'response')

2124
2092
2094
2277
1981
1997
1959
2051
2047
1984
[0.014000000000000002, 0.031, 0.049, 0.066, 0.084, 0.101, 0.118, 0.13620000000000002, 0.1532, 0.17120000000000002, 0.1882, 0.2062, 0.2234, 0.2408, 0.25880000000000003, 0.27620000000000006, 0.29419999999999996, 0.31120000000000003, 0.3286, 0.34659999999999996, 0.36379999999999996, 0.382, 0.3992, 0.4168, 0.4346, 0.4518, 0.46979999999999994, 0.4875999999999999, 0.5058, 0.523, 0.5408000000000001, 0.5594000000000001, 0.5768000000000001, 0.5958, 0.6136, 0.6315999999999999, 0.6502000000000001, 0.668, 0.6868, 0.7048, 0.7234, 0.7416, 0.7598, 0.7784, 0.7956000000000001, 0.8143999999999998, 0.8320000000000001, 0.8503999999999999, 0.869, 0.8867999999999998, 0.9052, 0.9231999999999999, 0.9416, 0.9592, 0.9776, 0.9954000000000001, 1.0137999999999998, 1.0318, 1.0492000000000001, 1.068, 1.0858, 1.1038000000000001, 1.1219999999999999, 1.1406, 1.1583999999999999, 1.176, 1.1954, 1.2132, 1.231, 1.2494, 1.2675999999999998, 1.2861999999999998, 1.3039999999999

In [5]:
# fig,axs = plt.subplots(10,1,figsize=(6,9))
# plt.subplots_adjust(top = 0.95,bottom = 0.05)
# figraw,axsraw = plt.subplots(10,1,figsize=(6,9))
# plt.subplots_adjust(top = 0.95,bottom = 0.05)

# fig.suptitle('Filtered Manual Tuned Responses for Squash Ball')
# figraw.suptitle('Unfiltered Manual Tuned Responses for Squash Ball')

# fig2,ax2 = plt.subplots()
# fig3,ax3 = plt.subplots()
# fig2.suptitle('Filtered Average of Manually Tuned Squash Ball Responses')
# fig3.suptitle('Unfiltered Average of Manually Tuned Squash Ball Responses')
# ax2.set_xlabel('Time [s]')
# ax2.set_ylabel('Distance [cm]')
# ax3.set_xlabel('Time [s]')
# ax3.set_ylabel('Distance [cm]')

# time_1 = []
# time_2 = []
# time_3 = []
# time_4 = []
# time_5 = []
# distance_1 = []
# distance_2 = []
# distance_3 = []
# distance_4 = []
# distance_5 = []
# time_ave = []
# dist_ave = []

# sos = signal.butter(4, 1000, 'lp', fs=50000, output='sos')

# for i in range(1,11):
#     df = pd.read_csv('SQUASH_MAT_MAN_{}.csv'.format(i))
    
#     time = (df['time']-5000)/1000
#     time = time[0:1918]
#     distance = df['distance']
#     distance = distance[0:1918]
#     #print(len(distance))
#     filtered = signal.sosfiltfilt(sos,distance)
    
#     axs[i-1].plot(time,filtered)
#     axs[i-1].axhline(15,color='r',linestyle='--',label = 'setpoint')
#     axs[i-1].set_xlabel('Time [s]')
#     axs[i-1].set_ylabel('Distance [cm]')
#     axs[i-1].legend(loc=4)
    
#     axsraw[i-1].plot(time,distance)
#     axsraw[i-1].set_xlabel('Time [s]')
#     axsraw[i-1].set_ylabel('Distance [cm]')
#     axsraw[i-1].axhline(15,color='r',linestyle='--',label = 'setpoint')
#     axsraw[i-1].legend(loc=4)    
#     time_name =f"time_{i}"
#     globals()[time_name]=((time[0:1918].to_numpy()))
#     distance_name = f"distance_{i}"
#     globals()[distance_name]=distance[0:1918].to_numpy()
#     #print(distance[1530])

# for i in range(0,1918):
#     time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
#     dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)

# filtered = signal.sosfiltfilt(sos,dist_ave)
# ax2.plot(time_ave,filtered)
# ax3.plot(time_ave,dist_ave)

In [24]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)
figraw,axsraw = plt.subplots()

figz,axz = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)

fig.suptitle('Individual Manually Tuned Responses for Squash Ball',size=20)
figraw.suptitle('Unfiltered Manually Tuned Responses for Squash Ball',size=20)
figz.suptitle('Motor Angle for Manually Tuned Squash Ball Response',size=20)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Manually Tuned Squash Ball Responses',fontsize=15)
#fig3.suptitle('Unfiltered Average of Manually Tuned Squash Ball Responses')
ax2.set_xlabel('Time [s]',fontsize=15)
ax2.set_ylabel('Distance [cm]',fontsize=15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
time_6 = []
time_7 = []
time_8 = []
time_9 = []
time_10 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
distance_6 = []
distance_7 = []
distance_8 = []
distance_9 = []
distance_10 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
angle_6 = []
angle_7 = []
angle_8 = []
angle_9 = []
angle_10 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(4, 800, 'lp', fs=50000, output='sos')
sosang = signal.butter(4, 10000, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
ticksang = np.linspace(-40,40,17)
END = 1200

lines = ['solid','dashed','dotted','dashdot','solid']
for i in range(1,6):
    df = pd.read_csv('SQUASH_MAT_MAN_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sosang,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i),linestyle=lines[i-1])
    axs.set_xlabel('Time [s]',fontsize=15)
    axs.set_ylabel('Distance [cm]',fontsize=15)

    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]',fontsize=15)
    axsraw.set_ylabel('Distance [cm]',fontsize=15)
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axz.set_xlabel('Time [s]',fontsize=15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize=15)


    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()

axz.set_yticks(ticksang)
    
for i in range(1,6):
    df = pd.read_csv('SQUASH_MAT_MAN_{}.csv'.format(i+5))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sosang,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i+5),linestyle=lines[i-1])
    axs.set_xlabel('Time [s]',fontsize=15)
    axs.set_ylabel('Distance [cm]',fontsize=15)


    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]',fontsize=15)
    axsraw.set_ylabel('Distance [cm]',fontsize=15)
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i+5),linestyle=lines[i-1])
    axz.set_xlabel('Time [s]',fontsize=15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize=15)


    axz.set_yticks(ticksang)
    time_name =f"time_{i+5}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i+5}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')
axs.grid(which='both')
axz.grid(which = 'both')
axs.legend(loc = 'right',fontsize=15)
axz.legend(loc = 'right',fontsize=15)

for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i],time_6[i],time_7[i],time_8[i],time_9[i],time_10[i]])/10)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i],distance_6[i],distance_7[i],distance_8[i],distance_9[i],distance_10[i]])/10)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<2.5 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = "darkslategrey")
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right',fontsize=15)
ticks = np.linspace(0,23,24)
ax2.set_yticks(ticks)
#ax3.plot(time_ave,dist_ave,label = 'response')

2124
2092
2094
2277
1981
1997
1959
2051
2047
1984
[0.014000000000000002, 0.031100000000000006, 0.0493, 0.0665, 0.08459999999999998, 0.1018, 0.1193, 0.1369, 0.15389999999999998, 0.17209999999999998, 0.1892, 0.2072, 0.2244, 0.24169999999999997, 0.25980000000000003, 0.2771, 0.29519999999999996, 0.31229999999999997, 0.3295, 0.3476, 0.36479999999999996, 0.38309999999999994, 0.4002, 0.41789999999999994, 0.4356, 0.4528, 0.471, 0.48840000000000006, 0.5067, 0.524, 0.5417000000000001, 0.5601, 0.5775, 0.5962000000000001, 0.614, 0.6319999999999999, 0.6504000000000001, 0.6681, 0.6868000000000001, 0.7046, 0.7232, 0.741, 0.7592, 0.7775000000000001, 0.7950000000000002, 0.8135999999999999, 0.8311999999999999, 0.8499000000000001, 0.868, 0.8859999999999998, 0.9043000000000001, 0.9222999999999999, 0.9407, 0.9585000000000001, 0.9769, 0.9949000000000001, 1.0131999999999999, 1.0313, 1.0492, 1.0678999999999998, 1.0859, 1.1043, 1.1221999999999999, 1.1406, 1.1588, 1.1764999999999999, 1.1957, 1.2135000000000002,